In [1]:
from transformers import pipeline
from PIL import Image
import os

c:\Users\sreer\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
pipe = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf")
image = Image.open('../images/akash_001.jpg')
depth = pipe(image)["depth"]

Device set to use cuda:0


In [9]:
def create_depth_map(image):
    depth = pipe(image)["depth"]
    depth_map = depth.resize(image.size)
    return depth_map

In [10]:
def process_directory_images(directory_path, save=False):
    for filename in os.listdir(directory_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            img_path = os.path.join(directory_path, filename)
            img = Image.open(img_path)
            depth_map = create_depth_map(img)
            if save:
                depth_map.save(os.path.join(directory_path, f"depth_map_{filename}"))

In [11]:
process_directory_images('../images', save=True)

## Depth Analysis Based Table Detection

In [2]:
from ultralytics import YOLO
import cv2  
import numpy as np

In [3]:
def annotate_frame(frame, yolo_results):
    """Draw YOLO keypoints on frame"""
    annotated = frame.copy()
    for r in yolo_results:
        # check if keypoints exist and have at least 4 keypoints
        if hasattr(r, 'keypoints') and r.keypoints is not None:
            keypoints = r.keypoints.cpu().numpy()[0]  # first detection, shape (num_keypoints, 3)
            if keypoints.shape[0] >= 4:
                for kp in keypoints:
                    x, y = kp[:2]  # take only x, y, ignore confidence
                    cv2.circle(annotated, (int(x), int(y)), 5, (0, 255, 0), -1)
            else:
            # fallback to bounding box if not enough keypoints
                if hasattr(r, 'boxes') and r.boxes is not None:
                    box = r.boxes.xyxy.cpu().numpy()[0]  # [x1, y1, x2, y2]
                    x1, y1, x2, y2 = map(int, box)
                    cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 0, 255), 2)
        else:
            # fallback to bounding box if keypoints attribute missing or None
            if hasattr(r, 'boxes') and r.boxes is not None:
                box = r.boxes.xyxy.cpu().numpy()[0]  # [x1, y1, x2, y2]
                x1, y1, x2, y2 = map(int, box)
                cv2.rectangle(annotated, (x1, y1), (x2, y2), (0, 0, 255), 2)
    return annotated


In [ ]:
yolo_model = YOLO("../yolov8-pose/runs/pose/train5/weights/best.pt")
pipe = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small-hf")
cap = cv2.VideoCapture("../Videos/rallies_01.mp4")   # Replace with "video.mp4" for a file
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # --- Depth estimation ---
    # Convert OpenCV frame to PIL
    pil_frame = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Run depth estimation
    depth_result = pipe(pil_frame)  # returns a dict

    # Get depth map (as PIL image)
    depth_pil = depth_result["depth"]
    depth_map = np.array(depth_pil).astype(np.float32)

    # Normalize and apply colormap for visualization
    depth_norm = cv2.normalize(depth_map, None, 0, 255, cv2.NORM_MINMAX)
    depth_color = cv2.applyColorMap(depth_norm.astype(np.uint8), cv2.COLORMAP_MAGMA)



    # --- YOLO pose detection ---
    yolo_results = yolo_model(frame)  # frame still BGR

    # --- Annotate frame with YOLO keypoints ---
    annotated_frame = annotate_frame(frame, yolo_results)

    # Combine annotated frame and depth map side by side
    # combined = np.hstack((annotated_frame, depth_color))

    # Show frame
    cv2.imshow("YOLO + Depth", annotated_frame)
    
    # Save frame

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Device set to use cuda:0



0: 384x640 1 table, 144.0ms
Speed: 3.4ms preprocess, 144.0ms inference, 201.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 table, 20.4ms
Speed: 2.0ms preprocess, 20.4ms inference, 5.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 16.4ms
Speed: 2.2ms preprocess, 16.4ms inference, 2.7ms postprocess per image at shape (1, 3, 384, 640)


IndexError: index 0 is out of bounds for axis 0 with size 0

: 